In [1]:
import sqlite3
import pandas as pd

# Path to your CSV file
csv_file_path = 'data/github_final.csv'
# Name of the SQLite database
sqlite_db_path = 'data/github_final.db'

# Read CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

# Connect to SQLite database
conn = sqlite3.connect(sqlite_db_path)

# Write the data to SQLite table
df.to_sql('github_final', conn, if_exists='replace', index=False)

# Commit changes and close the connection
conn.commit()
conn.close()

print("CSV data has been imported into the SQLite database.")

CSV data has been imported into the SQLite database.


In [2]:
df.head()

,name,full_name,topics,description,created_at,stargazers_count,forks_count,license,days_since_created,avg_forks_per_day,avg_stars_per_day,categories
0,langchain,langchain-ai/langchain,no topic,🦜🔗 Build context-aware reasoning applications,2022-10-17,81425,12434,MIT License,540,23.0,151.0,Applications
1,ChatGPT-Next-Web,ChatGPTNextWeb/ChatGPT-Next-Web,"chatgpt, cross-platform, desktop, fe, gemini, ...",A cross-platform ChatGPT/Gemini UI (Web / PWA ...,2023-03-10,66548,54259,MIT License,396,137.0,168.0,Applications
2,awesome-machine-learning,josephmisiti/awesome-machine-learning,no topic,A curated list of awesome Machine Learning fra...,2014-07-15,63304,14447,Other,3556,4.0,18.0,Lists
3,awesome-cpp,fffaraz/awesome-cpp,"awesome, awesome-list, c, c-plus-plus, cpp, cp...",A curated list of awesome C++ (or C) framework...,2014-07-17,55101,7613,MIT License,3554,2.0,16.0,Lists
4,llama.cpp,ggerganov/llama.cpp,"ggml, llama",LLM inference in C/C++,2023-03-10,54581,7703,MIT License,396,19.0,138.0,Tutorials


In [3]:
from langchain_community.utilities.sql_database import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///data/github_final.db")

In [4]:
from langchain.chains import create_sql_query_chain
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

prompt_write_query=PromptTemplate.from_template(
    template="""You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
    Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
    Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
    
    Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
    Pay attention to use date(\'now\') function to get the current date, if the question involves "today".
    
    Use the following format:
    
    Question: Question here
    SQLQuery: SQL Query to run
    SQLResult: Result of the SQLQuery
    Answer: Final answer here
    
    Only use the following tables:
    {table_info}
    
    Question: {input}""")

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

chain = create_sql_query_chain(
    llm, 
    db,
    prompt_write_query,
    k=5)

response = chain.invoke({"question": "Which project has the most forks? List top 10"})
print(response)

SELECT "name", "forks_count"
FROM github_final
ORDER BY "forks_count" DESC
LIMIT 10;


In [5]:
db.run(response)

"[('ChatGPT-Next-Web', 54259), ('generative-ai-for-beginners', 21598), ('awesome-machine-learning', 14447), ('langchain', 12434), ('Flowise', 11784), ('llama.cpp', 7703), ('awesome-cpp', 7613), ('private-gpt', 6804), ('chatgpt-on-wechat', 6622), ('lobe-chat', 5941)]"

In [6]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm, db)
chain = write_query | execute_query
chain.invoke({"question": "Which project has the most forks? List  top 10"})

"[('ChatGPT-Next-Web', 54259), ('generative-ai-for-beginners', 21598), ('awesome-machine-learning', 14447), ('langchain', 12434), ('Flowise', 11784), ('llama.cpp', 7703), ('awesome-cpp', 7613), ('private-gpt', 6804), ('chatgpt-on-wechat', 6622), ('lobe-chat', 5941)]"

In [9]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

prompt_query_answer = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

answer = prompt_query_answer | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

chain.invoke({"question": "List top 10 which the highest number of forks"})

'The top 10 repositories with the highest number of forks are:\n\n1. ChatGPT-Next-Web with 54,259 forks\n2. generative-ai-for-beginners with 21,598 forks\n3. awesome-machine-learning with 14,447 forks\n4. langchain with 12,434 forks\n5. Flowise with 11,784 forks\n6. llama.cpp with 7,703 forks\n7. awesome-cpp with 7,613 forks\n8. private-gpt with 6,804 forks\n9. chatgpt-on-wechat with 6,622 forks\n10. lobe-chat with 5,941 forks'